In [1]:
# syft absolute
import syft as sy
from syft import ActionObject
from syft.client.syncing import compare_states
from syft.client.syncing import resolve

In [2]:
node_low = sy.orchestra.launch(
    name="test_l",
    node_side_type="low",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

node_high = sy.orchestra.launch(
    name="test_h",
    node_side_type="high",
    dev_mode=True,
    reset=True,
    local_db=True,
    n_consumers=1,
    create_producer=True,
)

Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8a1c04544655402190588aec30079bc3.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=2afcf35284274a2798b046f05f7ba362 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/8212e6797fde4c3fba4fc53ab555a886.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=1 image_uid=a0030cd8c9744b4894e58c3114ad2167 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!


In [3]:
client_low = node_low.login(email="info@openmined.org", password="changethis")
client_high = node_high.login(email="info@openmined.org", password="changethis")

Logged into <test_l: Low side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Logged into <test_h: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

In [4]:
client_low.register(
    email="newuser@openmined.org", name="John Doe", password="pw", password_verify="pw"
)
client_low_ds = node_low.login(email="newuser@openmined.org", password="pw")

Logged into <test_l: Low side Domain> as <newuser@openmined.org>


# create datasets

In [5]:
# third party
import numpy as np

In [6]:
mock_high = np.array([10, 11, 12, 13, 14])
private_high = np.array([15, 16, 17, 18, 19])

dataset_high = sy.Dataset(
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_high,
            data=private_high,
            shape=private_high.shape,
            mock_is_real=True,
        )
    ],
)

client_high.upload_dataset(dataset_high)

100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  9.29it/s]

Uploading: numpy-data


SyftSuccess: Dataset uploaded to 'test_h'. To see the datasets uploaded by a client on this node, use command `[your_client].datasets`

In [7]:
mock_low = np.array([0, 1, 2, 3, 4])  # do_high.mock
# private_low = np.array([5, 6, 7, 8, 9])  # AOEmpty? create new type AO

dataset_low = sy.Dataset(
    id=dataset_high.id,
    name="my-dataset",
    description="abc",
    asset_list=[
        sy.Asset(
            name="numpy-data",
            mock=mock_low,
            data=ActionObject.empty(data_node_id=client_high.id),
            shape=mock_low.shape,
            mock_is_real=True,
        )
    ],
)

res = client_low.upload_dataset(dataset_low)

  0%|                                                                                             | 0/1 [00:00<?, ?it/s]

Uploading: numpy-data


100%|█████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 30.46it/s]


In [8]:
dataset_storage_permissions = node_low.python_node.get_service(
    "datasetservice"
).stash.partition.storage_permissions
dataset_id = client_low.datasets[0].id

assert dataset_storage_permissions[dataset_id] == {node_low.python_node.id}

# Data Scientist: make requests

In [9]:
@sy.syft_function()
def sub_function(data) -> float:
    print("executing sub function")
    return data.mean()

client_low_ds.code.submit(sub_function)

SyftSuccess: Syft function 'sub_function' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftSuccess: User Code Submitted

In [10]:
data_low = client_low_ds.datasets[0].assets[0]


@sy.syft_function_single_use(data=data_low)
def compute_mean(domain, data) -> float:
    print("executing main function")
    job = domain.launch_job(sub_function, data=data)
    return job.result

res = compute_mean(data=data_low.mock, blocking=True)

SyftSuccess: Syft function 'compute_mean' successfully created. To add a code request, please create a project using `project = syft.Project(...)`, then use command `project.create_code_request`.

SyftInfo: Creating a node with n_consumers=2 (the default value)
Staging Protocol Changes...
SQLite Store Path:
!open file:///var/folders/pn/f6xkq7mx683g5jkyt91gqyzw0000gn/T/09223b3985a444479e6f9732498e882b.sqlite

Creating default worker image with tag='local-dev'
Building default worker image with tag=local-dev
Setting up worker poolname=default-pool workers=2 image_uid=59560c68b2f348ec8da8cac629ce3b59 in_memory=True
Created default worker pool.
Data Migrated to latest version !!!
Logged into <ephemeral_node_compute_mean_6761: High side Domain> as <info@openmined.org>


SyftWarning: You are using a default password. Please change the password using `[your_client].me.set_password([new_password])`.

Approving request for domain ephemeral_node_compute_mean_6761
executing main function
SyftInfo: Landing the ephmeral node...


In [11]:
res.get()

SyftError: SyftError: This object is not resolved yet.

In [12]:
client_low_ds.code.request_code_execution(compute_mean)

SyftInfo: Node Landed!


syft.service.request.request.Request

In [13]:
client_low_ds.code.compute_mean(data=data_low)

SyftError: Execution denied, Please wait for the code to be approved

## Data Owner: Sync to high side

In [14]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

In [15]:
low_state

syft.service.sync.sync_state.SyncState

In [16]:
assert (
    set(low_state.objects.keys())
    == set(low_state.permissions.keys())
    == set(low_state.storage_permissions.keys())
)

In [17]:
diff_state = compare_states(low_state, high_state)

In [18]:
diff_state

syft.service.sync.diff_state.NodeDiff

In [19]:
diff_state.hierarchies[0].diffs

[ObjectDiff[UserCodeStatusCollection](#966cc70272784eb98a4888b56b252bda),
 ObjectDiff[UserCode](#bae0bd2390f64a7e8d2b2e289f885a66)]

In [20]:
resolved_state_low, resolved_state_high = resolve(diff_state, decision="low")

LOW SIDE STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #bae0bd2390f64a7e8d2b2e289f885a66:
service_func_name: sub_function
input_owners: [
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]
worker_pool_name: default-pool

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #966cc70272784eb98a4888b56b252bda:
    approved: False
    status_dict: {
    ‎ ‎ ‎ ‎ node_id=<UID: 8a1c04544655402190588aec30079bc3>
        verify_key=1c0590b894d19530970da6bf09bb302afa3e473a4cccdacbace0e20223ee3367
        node_name='test_l': (<UserCodeStatus.PENDING: 'pending'>, '')
    }



HIGH SIDE STATE:

No high side changes.

Decision: Syncing 2 objects from low side


LOW SIDE STATE:

―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #72afd3182a3446a7b6fc5e633e4de337:
service_func_name: compu

In [21]:
print("Resolved state low side")
print(resolved_state_low)
print()
print("Resolved state high side")
print(resolved_state_high)

Resolved state low side
ResolvedSyncState(
  create_objs=[],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)

Resolved state high side
ResolvedSyncState(
  create_objs=[{NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}, syft.service.code.user_code.UserCode, {NodeIdentity <name=test_l, id=8a1c0454, 🔑=1c0590b8>: (<UserCodeStatus.PENDING: 'pending'>, '')}, syft.service.code.user_code.UserCode, syft.service.request.request.Request],
  update_objs=[],
  delete_objs=[]
  new_permissions=[]
)


In [22]:
assert len(resolved_state_high.new_permissions) == 0

new_objs = resolved_state_high.create_objs
assert {o.id for o in new_objs} == {
    sp.uid for sp in resolved_state_high.new_storage_permissions
}

In [23]:
client_low.apply_state(resolved_state_low)

SyftSuccess: Synced 0 items

In [24]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 5 items

In [25]:
request_storage_permissions = node_high.python_node.get_service(
    "requestservice"
).stash.partition.storage_permissions
request_id = client_high.requests[0].id
node_high_id = node_high.python_node.id

assert node_high_id in request_storage_permissions[request_id]

# Data Owner: Run code high-side

In [26]:
data_high = client_high.datasets[0].assets[0]

In [27]:
job_high = client_high.code.compute_mean(data=data_high, blocking=False)
display(job_high)

```python
class Job:
    id: UID = 90323a2fb5a142b28f7a3c2117491ae6
    status: created
    has_parent: False
    result: syft.service.action.action_data_empty.ObjectNotReady
    logs:

0 
    
```

In [28]:
# wait for the result
job_high.wait().get()

14/03/24 16:29:19 FUNCTION LOG (90323a2fb5a142b28f7a3c2117491ae6): executing main function
14/03/24 16:29:21 FUNCTION LOG (43e5309085de4abeb02070a94cdc6419): executing sub function


17.0

In [29]:
job_info = job_high.info(public_metadata=True, result=True)

request = client_high.requests[0]
result_obj = job_high.result

In [30]:
job_info

syft.service.job.job_stash.JobInfo

In [31]:
# syft absolute
from syft import SyftError
from syft import SyftSuccess

# Accepting the result directly gives an error
accept_res = request.accept_by_depositing_result(result_obj)
assert isinstance(accept_res, SyftError)

In [32]:
accept_res = request.accept_by_depositing_result(job_info)

assert isinstance(accept_res, SyftSuccess)
accept_res

Approving request for domain test_h
returning existing job
setting permission
None
None
Approving request for domain test_h
ADDING PERMISSION [READ: dd66ce8d17694336991728ebaf903e94 as b7aff973444aa2ede65e201d9a531d1a94fbeaa8d26bad78ef96d0ae122bd240] dd66ce8d17694336991728ebaf903e94
Job(90323a2fb5a142b28f7a3c2117491ae6) Setting new result dd66ce8d17694336991728ebaf903e94 -> dd66ce8d17694336991728ebaf903e94


SyftSuccess: Request submitted for updating result.

In [33]:
# Need to refresh Job because we overwrite result
job_high = client_high.code.compute_mean.jobs[0]

action_store_high = node_high.python_node.get_service("actionservice").store
blob_store_high = node_high.python_node.get_service(
    "blobstorageservice"
).stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_high.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_high.permissions[job_high.result.syft_blob_storage_entry_id]
)

In [34]:
assert action_store_high.storage_permissions[job_high.result.id.id] == {
    node_high.python_node.id
}

## Data Owner: Sync back to low-side

In [35]:
low_state = client_low.get_sync_state()
high_state = client_high.get_sync_state()

if isinstance(low_state, sy.SyftError):
    print("low")
    print(low_state.message)

if isinstance(high_state, sy.SyftError):
    print("high")
    print(high_state.message)

CLOSING DB
high
Exception calling sync._get_state. Traceback (most recent call last):
  File "/Users/eelco/dev/PySyft/packages/syft/src/syft/node/node.py", line 1264, in handle_api_call_with_unsigned_result
    result = method(context, *api_call.args, **api_call.kwargs)
  File "/Users/eelco/dev/PySyft/packages/syft/src/syft/service/service.py", line 365, in _decorator
    result = func(self, *args, **kwargs)
  File "/Users/eelco/dev/PySyft/packages/syft/src/syft/service/sync/sync_service.py", line 258, in _get_state
    new_state.add_objects(items, api=node.root_client.api)  # type: ignore
  File "/Users/eelco/dev/PySyft/packages/syft/src/syft/node/node.py", line 706, in root_client
    root_client.api.refresh_api_callback()
  File "/Users/eelco/dev/PySyft/packages/syft/src/syft/client/client.py", line 1138, in refresh_callback
    return self._fetch_api(self.credentials)
  File "/Users/eelco/dev/PySyft/packages/syft/src/syft/client/client.py", line 1132, in _fetch_api
    _api: SyftAP

In [37]:
client_high.verify_key

d311a667006cbb56614a062b2bbc7a733b5a4a8edd5293e0f34bb5c75f51277d

In [40]:
node_high.python_node.get_service("userservice").stash.partition.data

Exception: Error usually related to calling self.db.close()before last SQLiteBackingStore.__del__ gets called. Cannot operate on a closed database.

In [36]:
client_high.sync._get_state()

SyftError: As a `ServiceRole.GUEST`, you have no access to: sync._get_state

In [35]:
diff_state_2 = compare_states(low_state, high_state)

In [38]:
resolved_state_low, resolved_state_high = resolve(diff_state_2, share_private_objects=True)

LOW SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #17f94e53de4f443aa199d3bcd9f60dac:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]

    ―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODESTATUSCOLLECTION #86165494d8414222b39553c28dbb196a:
    status_dict: (<UserCodeStatus.PENDING: 'pending'>, '')

    ―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #a00dff07be724b26a7461622eef02f84:
    service_func_name: sub_function
    input_owners: [
    ]
    code_status: [
    ‎ ‎ ‎ ‎ Node: test_l, Status: pending
    ]



HIGH SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #17f94e53de4f443aa199d3bcd9f60dac:
service_func_name: compute_mean
input_owners

 high


Decision: Syncing 3 objects from high side


LOW SIDE STATE:

―――― DIFF ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

REQUEST #45f6cc62052e4e19a5ab858810b046fe:
node_uid: 8a1c04544655402190588aec30079bc3
changes: [
‎ ‎ ‎ ‎ syft.service.request.request.UserCodeStatusChange
]
history: [
]

    ―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #17f94e53de4f443aa199d3bcd9f60dac:
    service_func_name: compute_mean
    input_owners: [
    ‎ ‎ ‎ ‎ test_l
    ]
    code_status: [
    ‎ ‎ ‎ ‎ Node: test_l, Status: pending
    ]

        ―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

        USERCODE #a00dff07be724b26a7461622eef02f84:
        service_func_name: sub_function
        input_owners: [
        ]
        code_status: [
        ‎ ‎ ‎ ‎ Node: test_l, Status: pending
        ]



HIGH SIDE STATE:

―――― DIFF ――――――――――――――――――――――――――――

 high


Decision: Syncing 3 objects from high side


LOW SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #17f94e53de4f443aa199d3bcd9f60dac:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_l, Status: pending
]

    ―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

    USERCODE #a00dff07be724b26a7461622eef02f84:
    service_func_name: sub_function
    input_owners: [
    ]
    code_status: [
    ‎ ‎ ‎ ‎ Node: test_l, Status: pending
    ]



HIGH SIDE STATE:

―――― SAME ――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

USERCODE #17f94e53de4f443aa199d3bcd9f60dac:
service_func_name: compute_mean
input_owners: [
‎ ‎ ‎ ‎ test_l
]
code_status: [
‎ ‎ ‎ ‎ Node: test_h, Status: approved
]

    ―――― NEW ―――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――――

 

 high


Decision: Syncing 8 objects from high side




In [39]:
print(resolved_state_low)
print()
print(resolved_state_high)

ResolvedSyncState(
  create_objs=[syft.service.output.output_service.ExecutionOutput, syft.service.job.job_stash.Job, syft.service.job.job_stash.Job, Pointer:
17.0, syft.service.log.log.SyftLog, syft.service.log.log.SyftLog],
  update_objs=[{NodeIdentity <name=test_h, id=8212e679, 🔑=d311a667>: (<UserCodeStatus.APPROVED: 'approved'>, '')}, syft.service.request.request.Request],
  delete_objs=[]
  new_permissions=[[READ: 483473b7c80540caa3893ffddb6ec646 as f0570bb7c073974286d8ae6a4d50ac016aa1e045b86a1f17baefbcf622dc26bb], [READ: f32302766de84a12a8c195682b99d76b as f0570bb7c073974286d8ae6a4d50ac016aa1e045b86a1f17baefbcf622dc26bb], [READ: 9e3246b61d20482fa40a60bed51f9bc7 as f0570bb7c073974286d8ae6a4d50ac016aa1e045b86a1f17baefbcf622dc26bb], [READ: 1ba5c1907c85476588e7248121ea915b as f0570bb7c073974286d8ae6a4d50ac016aa1e045b86a1f17baefbcf622dc26bb], [READ: e0da0043ee7f451a8dd42877ed94005b as f0570bb7c073974286d8ae6a4d50ac016aa1e045b86a1f17baefbcf622dc26bb]]
)

ResolvedSyncState(
  create_obj

In [38]:
res = client_low.apply_state(resolved_state_low)
res

SyftSuccess: Synced 9 items

In [39]:
client_high.apply_state(resolved_state_high)

SyftSuccess: Synced 0 items

In [40]:
action_store_low = node_low.python_node.get_service("actionservice").store
blob_store_low = node_low.python_node.get_service("blobstorageservice").stash.partition
assert (
    f"{client_low_ds.verify_key}_READ"
    in action_store_low.permissions[job_high.result.id.id]
)
assert (
    f"{client_low_ds.verify_key}_READ"
    in blob_store_low.permissions[job_high.result.syft_blob_storage_entry_id]
)

In [41]:
result_storage_permissions = node_low.python_node.get_service(
    "actionservice"
).store.storage_permissions[job_high.result.id.id]
assert len(result_storage_permissions) == 1

log_storage_permissions = node_low.python_node.get_service(
    "logservice"
).stash.partition.storage_permissions[job_high.log_id]
assert len(log_storage_permissions) == 1

In [42]:
client_low.code.compute_mean.jobs[0].log_id

<UID: 681a2deba90f4a7daa01de3149a16f8c>

In [43]:
node_low.python_node.get_service("logservice").stash.partition.data[
    client_low.code.compute_mean.jobs[0].log_id
]

```python
class SyftLog:
  id: str = 681a2deba90f4a7daa01de3149a16f8c
  stdout: str = "executing main function
"
  stderr: str = ""

```

In [44]:
node_low.python_node.get_service("logservice").stash.partition.storage_permissions

{<UID: 681a2deba90f4a7daa01de3149a16f8c>: {<UID: 8a1c04544655402190588aec30079bc3>}, <UID: 2abd2ca427264114bbba5810dcf577c1>: {<UID: 8a1c04544655402190588aec30079bc3>}}

# Run code low

## Run

In [45]:
client_low_ds._fetch_api(client_low.credentials)

In [46]:
client_low_ds.code.compute_mean.jobs[0].result.get()

```python
class Job:
    id: UID = 43e11f3d77904d65870b7bba2cce8c8a
    status: completed
    has_parent: True
    result: 17.0
    logs:

0 Log 2abd2ca427264114bbba5810dcf577c1 not available
JOB COMPLETED
    
```

In [48]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

```python
Pointer
```
```python
class Job:
    id: UID = 054a73fd09ed4b2280c77bee98e16ffd
    status: completed
    has_parent: True
    result: 17.0
    logs:

0 Log 84ac701642d441808f1ae7147d78f2bc not available
JOB COMPLETED
    
```

In [ ]:
res_low.id == job_high.result.id

In [ ]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id.id
    == job_high.result.id.id
    == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [ ]:
res_low = client_low_ds.code.compute_mean(data=data_low)

res_low

In [ ]:
code = client_low_ds.code[0]

assert res_low.get() == private_high.mean()
assert (
    res_low.id.id
    == job_high.result.id.id
    == code.output_history[-1].output_ids[0].id.id
)
assert job_high.result.syft_blob_storage_entry_id == res_low.syft_blob_storage_entry_id

In [ ]:
private_high.mean()

In [ ]:
job_low = client_low_ds.code.compute_mean(data=data_low, blocking=False)
job_low

In [ ]:
job_low.wait().get()

In [ ]:
job_low.logs()

In [ ]:
assert job_low.id == job_high.id
assert job_low.result.id == job_high.result.id
assert (
    job_low.result.syft_blob_storage_entry_id
    == job_high.result.syft_blob_storage_entry_id
)